In [1]:
image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\322.png'

In [5]:
import cv2
import numpy as np

drawing = False
top_left_pt, bottom_right_pt = None, None

def draw_rectangle(event, x, y, flags, param):
    global drawing, top_left_pt, bottom_right_pt

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        top_left_pt = (x, y)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        bottom_right_pt = (x, y)
        cv2.rectangle(img, top_left_pt, bottom_right_pt, (0, 0, 255), 2)
        cv2.imshow('Image', img)

        # Process the region inside the rectangle with Sobel operator
        apply_sobel(top_left_pt, bottom_right_pt)

def apply_sobel(top_left, bottom_right):
    roi = img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    
    combined_sobel = np.sqrt(sobel_x**2 + sobel_y**2)
    normalized_sobel = cv2.normalize(combined_sobel, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    # Display the Sobel transformed image
    cv2.imshow('Sobel Transformed', normalized_sobel)

img = cv2.imread(image_path)
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', draw_rectangle)

while True:
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
